In [ ]:
# instalar libreria
#!pip install "git+https://github.com/C1587S/ACO-TSP#egg=ant-colony&subdirectory=src" &> /dev/null
from IPython.display import Image

## Implementación - Proyecto final

El propósito de este _notebook_ es presentar la implementación de la librería [ACO-TSP](https://github.com/C1587S/ACO-TSP/) utilizando un el conjunto de datos 400 de ciudades de China disponible en la base de datos _National Traveling Salesman Problems_ de la Universidad de Waterloo, disponible [aquí](https://www.math.uwaterloo.ca/tsp/world/countries.html). Esta implementación se realiza en forma de _pipeline_ para ser ejecutada en [`kale`](https://github.com/kubeflow-kale/kale). 

En particular, se siguen los siguientes pasos:

In [ ]:
Image("imgs/pipeline.png")

In [ ]:
import ant_colony as ac
import json
from multiprocessing import cpu_count

In [ ]:
## parametros del algoritmo
data_path = '../datasets/ch71009.tsp'
init_node_ex = 0
hp_trials = 10
save_hp = True
seed = 19519159
n_cities = 400
n_cities_hp = 50
n_cpu = cpu_count()

In [ ]:
# lectura de datos 

## 200 ciudades - coordenadas y grafo
coord_df = ac.read_coord_data(data_path, n_cities=n_cities, seed=seed, coord_df=True)
G = ac.read_coord_data(data_path, n_cities=n_cities, seed=seed, coord_df=False)
## 30 ciudades - grafo hp
G_hp = ac.read_coord_data(data_path, n_cities=n_cities_hp, seed=seed, coord_df=False)

In [5]:
# visualizar y guardar mapa
map_nodes = ac.plot_nodes_map(coord_df, save=True, save_as='map_nodes')
map_nodes

In [6]:
# optimizacion de hiper-parametros y guardar en disco
study = ac.optim_h_params_mp(G_hp,
                          init_node = init_node_ex,
                          trials = hp_trials, 
                          save = save_hp)

[I 2021-05-18 02:59:16,331] Using an existing study with name 'optimize_aco' instead of creating a new one.
[I 2021-05-18 02:59:19,405] Trial 10 finished with value: 2878.3820585989342 and parameters: {'n_ants': 188, 'max_iter': 100, 'rho': 0.6721218637800607, 'alpha': 3, 'beta': 4}. Best is trial 10 with value: 2878.3820585989342.
[I 2021-05-18 02:59:19,500] Trial 11 finished with value: 4687.969556480872 and parameters: {'n_ants': 2, 'max_iter': 100, 'rho': 0.6850398651294355, 'alpha': 3, 'beta': 4}. Best is trial 10 with value: 2878.3820585989342.
[I 2021-05-18 02:59:19,594] Trial 12 finished with value: 4473.551887828632 and parameters: {'n_ants': 2, 'max_iter': 100, 'rho': 0.6811966950436185, 'alpha': 3, 'beta': 4}. Best is trial 10 with value: 2878.3820585989342.
[I 2021-05-18 02:59:19,696] Trial 13 finished with value: 4358.294089929882 and parameters: {'n_ants': 2, 'max_iter': 100, 'rho': 0.6392019798897988, 'alpha': 3, 'beta': 4}. Best is trial 10 with value: 2878.382058598934

Hyper-parameters saved in ./best_hiper_params.db


In [16]:
# cargar mejores hp
best_params = ac.load_params('best_hiper_params.db')

In [21]:
# instanciar colony_multiw con mejores hp
colony_mw = ac.colony_multiw(G, 
                             init_node=0,  
                             n_workers = n_cpu,
                             verbose=True, k_verbose=10,
                              **best_params)

In [22]:
# solucionar el problema tsp con aco
colony_mw.solve_tsp()

iter: 0 / 100 - dist: 871.94
iter: 10 / 100 - dist: 871.94
iter: 20 / 100 - dist: 871.94
iter: 30 / 100 - dist: 871.94
iter: 40 / 100 - dist: 871.94
iter: 50 / 100 - dist: 871.94
iter: 60 / 100 - dist: 871.94
iter: 70 / 100 - dist: 871.94
iter: 80 / 100 - dist: 871.94
iter: 90 / 100 - dist: 871.94


------------------------------
Resumen:
	Nro. de hormigas: 160
	Iteraciones: 100
	Distancia: 871.9358616836037
	Nodo inicial: 0
	Ruta: [0, 86, 74, 83, 30, 40, 24, 51, 82, 72, 28, 36, 4, 46, 84, 2, 33, 9, 90, 42, 93, 18, 76, 1, 15, 10, 31, 79, 60, 89, 67, 59, 6, 57, 7, 44, 96, 48, 61, 81, 73, 21, 58, 80, 62, 75, 43, 12, 8, 78, 22, 70, 35, 63, 66, 16, 26, 32, 19, 11, 45, 17, 37, 52, 99, 88, 71, 85, 13, 23, 91, 54, 53, 3, 55, 77, 68, 98, 27, 64, 87, 38, 47, 95, 20, 41, 69, 34, 14, 39, 92, 65, 50, 94, 5, 29, 56, 49, 97, 25, 0]
------------------------------


In [29]:
# visualizar solución y guardar mapa como html
map_route = ac.plot_rout_map(coord_df, colony_mw.best_route, path_type='ants', 
                             save=True, save_as='map_aco_route')
map_route

In [24]:
# extraer mejor ruta
sln_r = colony_mw.best_route

In [25]:
# extraer mejor distancia
sln_d = colony_mw.best_dist

In [26]:
# diccionario con soluciones
sln_dic = {'route': sln_r, 'distance': float(sln_d)}

In [27]:
# guardar solución en disco
with open("aco_sln.json", "w") as outfile: 
    json.dump(sln_dic, outfile)
outfile.close()

## Referencias
- Kale: https://github.com/kubeflow-kale/kale
- National Travelling Salesman problmes: https://www.math.uwaterloo.ca/tsp/world/countries.html